In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os

from keras import backend

from keras.layers import AveragePooling2D, Input

from keras import layers, models

from keras.optimizers import Adam, SGD

from keras.callbacks import CSVLogger, LearningRateScheduler


import sys
sys.path.insert(0, './')
import DenseNet

# --! cifar-10

# --! main.lua 
# --! -netType densenet 
# --! -dataset cifar10 
# --! -batchSize 64 
# --! -nEpochs 300 
# --! -depth 100 
# --! -growthRate 12




Using TensorFlow backend.


In [4]:
# def main():
input = Input(shape=(32, 32, 3))
model = DenseNet.DenseNet(depth=50,
                            growthRate=12,
                            include_top=True, 
                            data_set='cifar10',
                            input_shape=(32,32,3),
                            pooling='avg',
                            classes=10)
# model.summary()
model.name



'densenet50'

# 또는

In [7]:
input = Input(shape=(32, 32, 3))
model = DenseNet.DenseNet(depth=[6,12,24,12],
                            growthRate=32,
                            include_top=True, 
                            data_set='imagenet',
                            input_shape=(32,32,3),
                            pooling='avg',
                            classes=10)
# model.summary()
model.name



'densenet113'

In [3]:
from keras.datasets import cifar10
import keras
from keras.utils import to_categorical
import numpy as np

In [4]:
# Load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# summarize dataset shape
print('Train', x_train.shape, y_train.shape)
print('Test', (x_test.shape, y_test.shape))

# summarize pixel values
print('Train', x_train.min(), x_train.max(), x_train.mean(), x_train.std())
print('Test', x_test.min(), x_test.max(), x_test.mean(), x_test.std())

Train (50000, 32, 32, 3) (50000, 1)
Test ((10000, 32, 32, 3), (10000, 1))
Train 0 255 120.70756512369792 64.1500758911213
Test 0 255 121.52915475260417 64.06097012299574


In [5]:
# Input image dimensions.
input_shape = x_train.shape[1:]

# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

subtract_pixel_mean = True
# # If subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

# Convert class vectors to binary class matrices.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)


# summarize dataset shape
print('Train', x_train.shape, y_train.shape)
print('Test', x_test.shape, y_test.shape)
# summarize pixel values
print('Train', x_train.min(), x_train.max(), x_train.mean(), x_train.std())
print('Test', x_test.min(), x_test.max(), x_test.mean(), x_test.std())


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 1)
Train (50000, 32, 32, 3) (50000, 10)
Test (10000, 32, 32, 3) (10000, 10)
Train -0.5500815 0.60786617 -3.2028754e-07 0.24895683
Test -0.5500815 0.60786617 0.0032215924 0.24854434


In [9]:
from keras.preprocessing.image import ImageDataGenerator

train_data_generator = ImageDataGenerator(
#         rescale=1./255,
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=False,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=False,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
#         validation_split=0.2
        )
train_data_generator.fit(x_train)
train_iterator = train_data_generator.flow(x_train, y_train, batch_size=64, shuffle=True)

# train_data_iterator = train_data_generator.flow(x_train, y_train, batch_size=64, subset='training')
# validation_data_iterator = train_data_generator.flow(x_train, y_train, batch_size=64, subset='validation')


# test_data_generator = ImageDataGenerator()
test_data_generator = ImageDataGenerator()
test_iterator = test_data_generator.flow(x_test, y_test, batch_size=64)

print(f'Batches {len(train_iterator)}, {len(test_iterator)}')

print(train_data_generator.mean)
print(train_data_generator.std)



# test_data_generator.fit(x_train)
# test_iterator = train_data_generator.flow(x_test, y_test, batch_size=64)
# 실패

Batches 782, 157
None
None


In [7]:
len(test_iterator)

313

In [10]:
set_start_lr = 1e-1
lr_reduce_per_epoch = 100
lr_reduce = 10

def lr_schedule(epoch):
    lr = 1e-1
    if epoch > 225:
        lr *= 1e-3
    elif epoch > 150:
        lr *= 1e-2
    print('epoch: ', epoch)
    print('Learning rate: ', lr)
    return lr


# sgd = SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

sgd = SGD(lr=lr_schedule(0), decay=1e-4, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])



csv_logger_path = os.getcwd()+f"/{model.name}\
(cifar10, \
batch_size=64, \
data_set norm and mean_pixel_subtract, \
rendom_erasing, \
data_generator)_log.csv"

if not os.path.isdir(os.path.dirname(csv_logger_path)):
    os.makedirs(os.path.dirname(csv_logger_path))
csv_logger = CSVLogger(csv_logger_path, append=True)
    
lr_scheduler = LearningRateScheduler(schedule=lr_schedule)

callbacks = [csv_logger, lr_scheduler]

model.fit_generator(train_iterator,
                    steps_per_epoch = len(train_iterator),
                    validation_data=test_iterator,
                    epochs=300, verbose=1, workers=4,
                    callbacks=callbacks)


epoch:  0
Learning rate:  0.1
Epoch 1/300
epoch:  0
Learning rate:  0.1
782/782 [==============================] - 57s 73ms/step - loss: 1.4728 - acc: 0.4588 - val_loss: 1.1633 - val_acc: 0.5988
Epoch 2/300
epoch:  1
Learning rate:  0.1
782/782 [==============================] - 52s 66ms/step - loss: 1.0125 - acc: 0.6407 - val_loss: 1.0605 - val_acc: 0.6347
Epoch 3/300
epoch:  2
Learning rate:  0.1
782/782 [==============================] - 52s 66ms/step - loss: 0.8299 - acc: 0.7053 - val_loss: 0.8706 - val_acc: 0.6892
Epoch 4/300
epoch:  3
Learning rate:  0.1
782/782 [==============================] - 51s 66ms/step - loss: 0.7091 - acc: 0.7520 - val_loss: 0.9717 - val_acc: 0.6667
Epoch 5/300
epoch:  4
Learning rate:  0.1
782/782 [==============================] - 52s 66ms/step - loss: 0.6320 - acc: 0.7799 - val_loss: 0.6323 - val_acc: 0.7778
Epoch 6/300
epoch:  5
Learning rate:  0.1
782/782 [==============================] - 52s 66ms/step - loss: 0.5743 - acc: 0.7983 - val_loss: 0.756

Epoch 50/300
epoch:  49
Learning rate:  0.1
782/782 [==============================] - 49s 62ms/step - loss: 0.1326 - acc: 0.9544 - val_loss: 0.4195 - val_acc: 0.8864
Epoch 51/300
epoch:  50
Learning rate:  0.1
782/782 [==============================] - 51s 65ms/step - loss: 0.1281 - acc: 0.9545 - val_loss: 0.4324 - val_acc: 0.8814
Epoch 52/300
epoch:  51
Learning rate:  0.1
782/782 [==============================] - 49s 63ms/step - loss: 0.1245 - acc: 0.9566 - val_loss: 0.4457 - val_acc: 0.8820
Epoch 53/300
epoch:  52
Learning rate:  0.1
782/782 [==============================] - 48s 62ms/step - loss: 0.1215 - acc: 0.9574 - val_loss: 0.4275 - val_acc: 0.8865
Epoch 54/300
epoch:  53
Learning rate:  0.1
782/782 [==============================] - 49s 63ms/step - loss: 0.1179 - acc: 0.9593 - val_loss: 0.4613 - val_acc: 0.8805
Epoch 55/300
epoch:  54
Learning rate:  0.1
782/782 [==============================] - 48s 62ms/step - loss: 0.1169 - acc: 0.9599 - val_loss: 0.4292 - val_acc: 0.886

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



782/782 [==============================] - 49s 63ms/step - loss: 0.0811 - acc: 0.9726 - val_loss: 0.4554 - val_acc: 0.8882
Epoch 86/300
epoch:  85
Learning rate:  0.1
782/782 [==============================] - 50s 64ms/step - loss: 0.0829 - acc: 0.9710 - val_loss: 0.4467 - val_acc: 0.8871
Epoch 87/300
epoch:  86
Learning rate:  0.1
782/782 [==============================] - 50s 64ms/step - loss: 0.0807 - acc: 0.9724 - val_loss: 0.4412 - val_acc: 0.8927
Epoch 88/300
epoch:  87
Learning rate:  0.1
744/782 [===========================>..] - ETA: 2s - loss: 0.0795 - acc: 0.9727

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



782/782 [==============================] - 47s 60ms/step - loss: 0.0478 - acc: 0.9839 - val_loss: 0.4411 - val_acc: 0.8981
Epoch 290/300
epoch:  289
Learning rate:  0.0001
782/782 [==============================] - 48s 62ms/step - loss: 0.0481 - acc: 0.9839 - val_loss: 0.4412 - val_acc: 0.8972
Epoch 291/300
epoch:  290
Learning rate:  0.0001
623/782 [======================>.......] - ETA: 9s - loss: 0.0473 - acc: 0.9840

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



782/782 [==============================] - 50s 63ms/step - loss: 0.0475 - acc: 0.9839 - val_loss: 0.4418 - val_acc: 0.8971
Epoch 294/300
epoch:  293
Learning rate:  0.0001
782/782 [==============================] - 49s 62ms/step - loss: 0.0470 - acc: 0.9838 - val_loss: 0.4398 - val_acc: 0.8972
Epoch 295/300
epoch:  294
Learning rate:  0.0001
140/782 [====>.........................] - ETA: 38s - loss: 0.0509 - acc: 0.9843

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
train_data_generator

In [ ]:
300*0.5

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
%matplotlib inline
SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))


In [13]:
scores = model.evaluate_generator(test_data_generator.flow(x_test, y_test, batch_size=64), verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

157/157 [==============================] - 2s 12ms/step
Test loss: 0.4421790162086487
Test accuracy: 0.8969
